# 1. Importing Dependencies

In [1]:
import cv2
import numpy as np 
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic  #holistic model
mp_drawing = mp.solutions.drawing_utils  #drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #color conversion BGR 2 RGB
    image.flags.writeable = False                   #image is no longer writeable
    results = model.process(image)                  #making the prediction
    image.flags.writeable = True                    #image is writeable again
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  #color vonversion RGB 2 BGR
    return image, results 

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)      #drawing face connections 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)       #drawing pose connections 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)  #drawing left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #drawing right hand connections


In [5]:
def draw_styled_landmarks(image, results):
    
    #drawing face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    
    #drawing pose connections 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    
    #drawing left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    
    #drawing right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 


# 3. Extracting Keypoints Values

In [6]:
def extract_landmarks(results) :
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)  
    return np.concatenate([pose, face, lh, rh])

# 4. Setting up Folders for Collection

In [7]:
#path for the exported data
DATA_PATH = os.path.join('MP_DATA')

#actions that we try to detect
actions = np.array(['hello', 'thanks', 'goodbye', 'howareyou', 'seeyoulater'])

#thirty videos worth of data
no_sequences = 30

#videos will be 30 frames in length
sequence_length = 30

In [15]:
for action in actions :
    for sequence in range(no_sequences) :
        try : 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except :
            pass

# 5. Collecting Keypoints for Training and Testing

In [16]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic :
    
    #looping through the actions
    for action in actions:
        #looping through the sequences
        for sequence in range(no_sequences):
            #looping through the sequence length 
            for frame_num in range(sequence_length):
        
                #reading the OpenCV feed
                ret, frame = cap.read()

                #making the prediction
                image, results = mediapipe_detection(frame, holistic)

                #drawing landmarks
                draw_styled_landmarks(image, results)
                
                #applying wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    #displaying frames 
                    cv2.imshow('OpenCV feed',image)
                    cv2.waitKey(3000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                    #displaying frames 
                    cv2.imshow('OpenCV feed',image)
                
                #exporting keypoints
                keypoints = extract_landmarks(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
        
                #breaking gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocessing Data and Creating Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'hello': 0, 'thanks': 1, 'goodbye': 2, 'howareyou': 3, 'seeyoulater': 4}

In [11]:
#loading the data
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(150, 30, 1662)

In [13]:
np.array(labels).shape

(150,)

In [14]:
X = np.array(sequences)

In [15]:
X.shape

(150, 30, 1662)

In [16]:
#one hot encoding y
y = to_categorical(labels).astype(int)

In [17]:
#shuffling the data
from random import shuffle
idx = list(range(len(X)))
shuffle(idx)
X_sh = X[idx]
y_sh = y[idx]

In [18]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Building and Training LSTM Neural Network

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [22]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [30]:
model.fit(X, y, epochs=200, callbacks=[tb_callback])

Epoch 1/200
5/5 [==============================] - 0s 61ms/step - loss: 1.7519 - categorical_accuracy: 0.1733
Epoch 2/200
5/5 [==============================] - 0s 60ms/step - loss: 1.6347 - categorical_accuracy: 0.2267
Epoch 3/200
5/5 [==============================] - 0s 60ms/step - loss: 1.6205 - categorical_accuracy: 0.1867
Epoch 4/200
5/5 [==============================] - 0s 60ms/step - loss: 1.5290 - categorical_accuracy: 0.3000
Epoch 5/200
5/5 [==============================] - 0s 60ms/step - loss: 1.4587 - categorical_accuracy: 0.3133
Epoch 6/200
5/5 [==============================] - 0s 62ms/step - loss: 1.3563 - categorical_accuracy: 0.3733
Epoch 7/200
5/5 [==============================] - 0s 61ms/step - loss: 1.2621 - categorical_accuracy: 0.4133
Epoch 8/200
5/5 [==============================] - 0s 60ms/step - loss: 1.2461 - categorical_accuracy: 0.4667
Epoch 9/200
5/5 [==============================] - 0s 61ms/step - loss: 1.2475 - categorical_accuracy: 0.3867
Epoch 10/2

5/5 [==============================] - 0s 67ms/step - loss: 0.1556 - categorical_accuracy: 0.9667
Epoch 148/200
5/5 [==============================] - 0s 68ms/step - loss: 0.1764 - categorical_accuracy: 0.9400
Epoch 149/200
5/5 [==============================] - 0s 66ms/step - loss: 0.1319 - categorical_accuracy: 0.9533
Epoch 150/200
5/5 [==============================] - 0s 68ms/step - loss: 0.1042 - categorical_accuracy: 0.9533
Epoch 151/200
5/5 [==============================] - 0s 71ms/step - loss: 0.0805 - categorical_accuracy: 0.9733
Epoch 152/200
5/5 [==============================] - 0s 66ms/step - loss: 0.0883 - categorical_accuracy: 0.9800
Epoch 153/200
5/5 [==============================] - 0s 67ms/step - loss: 0.0789 - categorical_accuracy: 0.9667
Epoch 154/200
5/5 [==============================] - 0s 65ms/step - loss: 0.0812 - categorical_accuracy: 0.9733
Epoch 155/200
5/5 [==============================] - 0s 65ms/step - loss: 0.0836 - categorical_accuracy: 0.9800
Epoch 

# 8. Making Predictions

In [31]:
res = model.predict(X_test)

1/1 [==============================] - 0s 31ms/step


In [36]:
actions[np.argmax(res[12])]

'goodbye'

In [37]:
actions[np.argmax(y_test[12])]

'goodbye'

# 9. Saving Weights

In [38]:
#this allows you to save your model 
model.save('bestactions.h5')

# 10. Testing in Real Time

In [39]:
from scipy import stats

In [40]:
colors = [(245,117,16), (117,245,16), (16,117,245),(80,245,117),(13,198,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [50]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Seting mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Reading feed
        ret, frame = cap.read()

        # Making detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Drawing landmarks
        #draw_styled_landmarks(image, results)
        
        # Prediction logic
        keypoints = extract_landmarks(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        # Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Showing to screen
        cv2.imshow('OpenCV Feed', image)

        # Breaking gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 30ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
hello
<class 'mediapipe.pyth

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
howareyou
<class 'mediapipe.python.solution_base.SolutionO